In [ ]:
!rm -rf /System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib

In [ ]:
!sudo pip uninstall matplotlib 
!sudo pip install matplotlib==1.5.1

In [ ]:
#!pip install -U protobuf
!pip install protobuf==3.20.*

In [ ]:
!pip install --upgrade tb-nightly

# Content Based KG

## Total Network

### Embedding 50

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import tensorflow.compat.v1 as tf
#import tensorflow as tf

tf.disable_v2_behavior()
tf.reset_default_graph()
#!pip install matplotlib==3.2
#tf.compat.v1.disable_v2_behavior()
# import tensorflow.python.ops.gen_logging_ops as logging_ops
from tensorflow.python.ops import gen_logging_ops
from tensorflow.python.framework import ops as _ops
from tensorflow.python.framework import ops as _ops
from tensorflow.python.ops import array_ops as _array_ops
from tensorflow.python.ops import gen_logging_ops as _gen_logging_ops
from tensorflow.python.ops import gen_summary_ops as _gen_summary_ops  # pylint: disable=unused-import
from tensorflow.python.ops import summary_op_util as _summary_op_util
from tensorflow.python.ops import summary_ops_v2 as _summary_ops_v2

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_dict_file = 'kb4rec_entity_new.csv'
        relation_dict_file = 'Relation2Id_new.csv'
        print('-----Loading entity dict-----')
        entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= ',')
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
       # print(self.entity_dict)
        self.n_entity = len(self.entity_dict)
        #print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= ',')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        self.n_relation = len(self.relation_dict)
        print('#relation: {}'.format(self.n_relation))
        with open('label_entity_embedding.csv','w') as file :
          write = csv.writer(file)
          write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'train_5.csv'
        validation_file = 'valid_5.csv'
        test_file = 'test_5.csv'
        print('-----Loading training triples-----')
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= '\t')
        self.training_triples = list(zip([self.entity_dict[h] for h in training_df[0]],
                                         [self.entity_dict[t] for t in training_df[2]],
                                         [self.relation_dict[r] for r in training_df[1]]))
        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        print('-----Loading validation triples-----')
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= '\t')
        self.validation_triples = list(zip([self.entity_dict[h] for h in validation_df[0]],
                                           [self.entity_dict[t] for t in validation_df[2]],
                                           [self.relation_dict[r] for r in validation_df[1]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= '\t')
        self.test_triples = list(zip([self.entity_dict[h] for h in test_df[0]],
                                     [self.entity_dict[t] for t in test_df[2]],
                                     [self.relation_dict[r] for r in test_df[1]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_embedding = tf.compat.v1.get_variable(name='entity',
                                                    shape=[kg.n_entity, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity)
            
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            head_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
        #print(n_batch)
        # print("merge!!!!!!!!!!!:",self.merge)
        # print("loss!!!!!!!!!!!:",self.loss)
        # print("train_op!!!!!!!!!!!:",self.train_op)
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
            
            #summary_writer.add_summary(tf.compat.v1.summary.merge(summary), global_step=self.global_step.eval(session=session))
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_embedding = self.entity_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        entity_norm = np.linalg.norm(entity_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)


        with open('entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_embedding)

        with open('relation_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)


    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())



def main():

    data_dir = ("/content/")
    embedding_dim = 50
    margin_value = 1.0
    score_func = "L1"
    batch_size =4800
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() # tf.summary.merge

-----Loading entity dict-----
-----Loading relation dict-----
#relation: 46
-----Loading training triples-----
#training triple: 94976
-----Loading validation triples-----
#validation triple: 95335
-----Loading test triples------
#test triple: 94976


Instructions for updating:
dim is deprecated, use axis instead


56788
-----Initializing tf graph-----
-----Initialization accomplished-----
-----Check norm-----
==============================[EPOCH 0]==============================
-----Start training-----
-----Constructing training batches-----
[0.525s] #triple: 93952/94976 triple_avg_loss: 1.021276
epoch loss: 100519.401
cost time: 0.525s
-----Finish training-----
-----Check norm-----
==============================[EPOCH 1]==============================
-----Start training-----
-----Constructing training batches-----
[0.475s] #triple: 93952/94976 triple_avg_loss: 0.600755
epoch loss: 61756.283
cost time: 0.475s
-----Finish training-----
-----Check norm-----
==============================[EPOCH 2]==============================
-----Start training-----
-----Constructing training batches-----
[0.464s] #triple: 91904/94976 triple_avg_loss: 0.219801
epoch loss: 46992.011
cost time: 0.464s
-----Finish training-----
-----Check norm-----
==============================[EPOCH 3]=============================

### Embedding 100

In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_dict_file = 'kb4rec_entity_new.csv'
        relation_dict_file = 'Relation2Id_new.csv'
        print('-----Loading entity dict-----')
        entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= ',')
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
       # print(self.entity_dict)
        self.n_entity = len(self.entity_dict)
        #print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= ',')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        self.n_relation = len(self.relation_dict)
        print('#relation: {}'.format(self.n_relation))
        with open('label_entity_embedding.csv','w') as file :
          write = csv.writer(file)
          write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'train_5.csv'
        validation_file = 'valid_5.csv'
        test_file = 'test_5.csv'
        print('-----Loading training triples-----')
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= '\t')
        self.training_triples = list(zip([self.entity_dict[h] for h in training_df[0]],
                                         [self.entity_dict[t] for t in training_df[2]],
                                         [self.relation_dict[r] for r in training_df[1]]))
        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        print('-----Loading validation triples-----')
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= '\t')
        self.validation_triples = list(zip([self.entity_dict[h] for h in validation_df[0]],
                                           [self.entity_dict[t] for t in validation_df[2]],
                                           [self.relation_dict[r] for r in validation_df[1]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= '\t')
        self.test_triples = list(zip([self.entity_dict[h] for h in test_df[0]],
                                     [self.entity_dict[t] for t in test_df[2]],
                                     [self.relation_dict[r] for r in test_df[1]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_embedding = tf.compat.v1.get_variable(name='entity',
                                                    shape=[kg.n_entity, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity)
            
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            head_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
        #print(n_batch)
        # print("merge!!!!!!!!!!!:",self.merge)
        # print("loss!!!!!!!!!!!:",self.loss)
        # print("train_op!!!!!!!!!!!:",self.train_op)
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
            #summary_writer.add_summary(tf.compat.v1.summary.merge(summary), global_step=self.global_step.eval(session=session))
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_embedding = self.entity_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        entity_norm = np.linalg.norm(entity_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)
        #print(entity_norm)
        #print(entity_embedding)
        # entity_embedding_list = []
        # entity_embedding_norm_list = []
        # entity_embedding_list.append(entity_embedding)
        # entity_embedding_norm_list.append(entity_norm)

        with open('entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_embedding)

        with open('relation_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)

        # with open('List_norm1.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(entity_norm)

        #return entity_embedding_list #, entity_embedding_norm_list
        
        #print('entity norm: {} relation norm: {}'.format(entity_norm, relation_norm))

    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())



def main():

    data_dir = ("/content/")
    embedding_dim = 100
    margin_value = 1.0
    score_func = "L1"
    batch_size =4800
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() 

## Trimmemd Network based on analysis

### Embedding 50

In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_dict_file = 'entity2id_case2 - entity2id_case2.csv'
        relation_dict_file = 'analyis_relation2Id_new.csv'
        print('-----Loading entity dict-----')
        entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= ',')
        
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
       # print(self.entity_dict)
        self.n_entity = len(self.entity_dict)
        print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= ',')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        #self.n_relation = len(self.relation_dict)
        self.n_relation = len(relation_df)
        
        print('#relation: {}'.format(self.n_relation))
        print(relation_df)
        with open('label_entity_embedding.csv','w') as file :
          write = csv.writer(file)
          write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'train_5_case2 - train_5_case2.csv'
        validation_file = 'val_5_case2 - val_5_case2.csv'
        test_file = 'test_5_case2 - test_5_case2.csv'
        print('-----Loading training triples-----')

       #training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None)
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= ',')
        self.training_triples = list(zip([self.entity_dict[h] for h in training_df[0]],
                                         [self.entity_dict[t] for t in training_df[2]],
                                         [self.relation_dict[r] for r in training_df[1]]))
        

          # self.training_triples = list(zip([self.entity_dict[h] for h in training_df[0]],
          #                                 [self.entity_dict[t] for t in training_df[2]],
          #                                 [self.relation_dict[r] for r in training_df[1]]))


        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        
        print('-----Loading validation triples-----')
        #validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None)
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= ',')
        self.validation_triples = list(zip([self.entity_dict[h] for h in validation_df[0]],
                                           [self.entity_dict[t] for t in validation_df[2]],
                                           [self.relation_dict[r] for r in validation_df[1]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        
        
        #test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None)
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= ',')
        self.test_triples = list(zip([self.entity_dict[h] for h in test_df[0]],
                                     [self.entity_dict[t] for t in test_df[2]],
                                     [self.relation_dict[r] for r in test_df[1]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_embedding = tf.compat.v1.get_variable(name='entity',
                                                    shape=[kg.n_entity, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity)
            
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            print('-----Initializing embedding -----')
            tf.compat.v1.global_variables_initializer()      
            head_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
      
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
          
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_embedding = self.entity_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        entity_norm = np.linalg.norm(entity_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)
        #print(entity_norm)
        #print(entity_embedding)
        # entity_embedding_list = []
        # entity_embedding_norm_list = []
        # entity_embedding_list.append(entity_embedding)
        # entity_embedding_norm_list.append(entity_norm)

        with open('entity_embedding_case2_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_embedding)

        with open('relation_embedding_case2_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)

        # with open('List_norm1.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(entity_norm)

        #return entity_embedding_list #, entity_embedding_norm_list
        
        #print('entity norm: {} relation norm: {}'.format(entity_norm, relation_norm))

    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())

    #     # tf.train.Saver를 이용해서 모델과 파라미터를 저장합니다.
    # def save(self, session, data_dir):
    #   SAVER_DIR = "model"
    #   saver = tf.compat.v1.train.Saver()
    #   checkpoint_path = os.path.join(SAVER_DIR, "model")
    #   ckpt = tf.compat.v1.train.get_checkpoint_state(SAVER_DIR)



def main():

    data_dir = ("/content/")
    embedding_dim = 50
    margin_value = 1.0
    score_func = "L1"
    batch_size =128
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() 

### Embedding 100

In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_dict_file = 'entity2id_case2 - entity2id_case2.csv'
        relation_dict_file = 'analyis_relation2Id_new.csv'
        print('-----Loading entity dict-----')
        entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= ',')
        
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
       # print(self.entity_dict)
        self.n_entity = len(self.entity_dict)
        print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= ',')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        #self.n_relation = len(self.relation_dict)
        self.n_relation = len(relation_df)
        
        print('#relation: {}'.format(self.n_relation))
        print(relation_df)
        with open('label_entity_embedding.csv','w') as file :
          write = csv.writer(file)
          write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'train_5_case2 - train_5_case2.csv'
        validation_file = 'val_5_case2 - val_5_case2.csv'
        test_file = 'test_5_case2 - test_5_case2.csv'
        print('-----Loading training triples-----')

       #training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None)
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= ',')
        self.training_triples = list(zip([self.entity_dict[h] for h in training_df[0]],
                                         [self.entity_dict[t] for t in training_df[2]],
                                         [self.relation_dict[r] for r in training_df[1]]))
        

          # self.training_triples = list(zip([self.entity_dict[h] for h in training_df[0]],
          #                                 [self.entity_dict[t] for t in training_df[2]],
          #                                 [self.relation_dict[r] for r in training_df[1]]))


        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        
        print('-----Loading validation triples-----')
        #validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None)
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= ',')
        self.validation_triples = list(zip([self.entity_dict[h] for h in validation_df[0]],
                                           [self.entity_dict[t] for t in validation_df[2]],
                                           [self.relation_dict[r] for r in validation_df[1]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        
        
        #test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None)
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= ',')
        self.test_triples = list(zip([self.entity_dict[h] for h in test_df[0]],
                                     [self.entity_dict[t] for t in test_df[2]],
                                     [self.relation_dict[r] for r in test_df[1]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_embedding = tf.compat.v1.get_variable(name='entity',
                                                    shape=[kg.n_entity, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity)
            
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            print('-----Initializing embedding -----')
            tf.compat.v1.global_variables_initializer()      
            head_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
        #print(n_batch)
        # print("merge!!!!!!!!!!!:",self.merge)
        # print("loss!!!!!!!!!!!:",self.loss)
        # print("train_op!!!!!!!!!!!:",self.train_op)
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
            #print("check2")
    
            # batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.merge],
            #                          feed_dict={self.triple_pos: batch_pos,
            #                                     self.triple_neg: batch_neg,
            #                                     self.margin: [self.margin_value] * len(batch_pos)})

            #print(summary)
            #summary = tf.compat.v1.summary.merge(summary)
            #print(self.global_step.eval(session=session))
            #summary_writer.add_summary(tf.compat.v1.summary.merge(summary), global_step=self.global_step.eval(session=session))
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_embedding = self.entity_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        entity_norm = np.linalg.norm(entity_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)
        #print(entity_norm)
        #print(entity_embedding)
        # entity_embedding_list = []
        # entity_embedding_norm_list = []
        # entity_embedding_list.append(entity_embedding)
        # entity_embedding_norm_list.append(entity_norm)

        with open('entity_embedding_case2_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_embedding)

        with open('relation_embedding_case2_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)

        # with open('List_norm1.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(entity_norm)

        #return entity_embedding_list #, entity_embedding_norm_list
        
        #print('entity norm: {} relation norm: {}'.format(entity_norm, relation_norm))

    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())

    #     # tf.train.Saver를 이용해서 모델과 파라미터를 저장합니다.
    # def save(self, session, data_dir):
    #   SAVER_DIR = "model"
    #   saver = tf.compat.v1.train.Saver()
    #   checkpoint_path = os.path.join(SAVER_DIR, "model")
    #   ckpt = tf.compat.v1.train.get_checkpoint_state(SAVER_DIR)



def main():

    data_dir = ("/content/")
    embedding_dim = 100
    margin_value = 1.0
    score_func = "L1"
    batch_size =128
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() 

# People Based KG

## Total Network

### Embedding 50

In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity_head = 0
        self.n_entity_tail = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_head_dict_file = 'entity2id_head.txt'
        entity_tail_dict_file = 'entity2id_tail.txt'
        relation_dict_file = 'relation2id.txt'
        print('-----Loading entity dict-----')
        #entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= '\t')
        entity_head_df = pd.read_table(os.path.join(self.data_dir, entity_head_dict_file), header=None, sep= '\t')
        entity_tail_df = pd.read_table(os.path.join(self.data_dir, entity_tail_dict_file), header=None, sep= '\t')        
        print(entity_head_df.head())
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        #self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
        self.entity_head_dict = dict(zip(entity_head_df[0], entity_head_df[1]))
        self.entity_tail_dict = dict(zip(entity_tail_df[0], entity_tail_df[1]))        
        self.n_entity_head = len(self.entity_head_dict)
        self.n_entity_tail = len(self.entity_tail_dict)
        #print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        
        print(self.n_entity_head)
        print(self.n_entity_tail)
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= '\t')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        print(self.relation_dict)
        self.n_relation = len(self.relation_dict)
        print(self.n_relation)
        print('#relation: {}'.format(self.n_relation))
        # with open('label_entity_embedding.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'imdb30_train - imdb30_train.csv'
        validation_file = 'imdb30_valid - imdb30_valid.csv'
        test_file = 'imdb30_test - imdb30_test.csv'
        print('-----Loading training triples-----')
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= ',')
        print(training_df.head())
        self.training_triples = list(zip([self.entity_head_dict[h] for h in training_df[0]],
                                         [self.entity_tail_dict[t] for t in training_df[1]],
                                         [self.relation_dict[r] for r in training_df[2]]))
        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        print('-----Loading validation triples-----')
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= ',')
        self.validation_triples = list(zip([self.entity_head_dict[h] for h in validation_df[0]],
                                           [self.entity_tail_dict[t] for t in validation_df[1]],
                                           [self.relation_dict[r] for r in validation_df[2]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= ',')
        self.test_triples = list(zip([self.entity_head_dict[h] for h in test_df[0]],
                                     [self.entity_tail_dict[t] for t in test_df[1]],
                                     [self.relation_dict[r] for r in test_df[2]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        print("---------check-------------------")
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_head_embedding = tf.compat.v1.get_variable(name='entity_head',
                                                    shape=[kg.n_entity_head, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            self.entity_tail_embedding = tf.compat.v1.get_variable(name='entity_tail',
                                                    shape=[kg.n_entity_tail, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            # self.entity_embedding = tf.compat.v1.get_variable(name='entity',
            #                                         shape=[kg.n_entity, self.embedding_dim],
            #                                         initializer=tf.random_uniform_initializer(minval=-bound,
            #                                                                                   maxval=bound))
            print("----------check2--------------------")
            tf.summary.histogram('self.entity_head_embedding.op.name', self.entity_head_embedding)
            tf.summary.histogram('self.entity_tail_embedding.op.name', self.entity_tail_embedding)
            
            #tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity_head)
            print(kg.n_entity_tail)           
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            #self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.entity_head_embedding = tf.nn.l2_normalize(self.entity_head_embedding, dim=1)
            self.entity_tail_embedding = tf.nn.l2_normalize(self.entity_tail_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            head_pos = tf.nn.embedding_lookup(self.entity_head_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_head_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_head_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_tail_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_head_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_tail_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
        #print(n_batch)
        # print("merge!!!!!!!!!!!:",self.merge)
        # print("loss!!!!!!!!!!!:",self.loss)
        # print("train_op!!!!!!!!!!!:",self.train_op)
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
            #print("check2")
    
            # batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.merge],
            #                          feed_dict={self.triple_pos: batch_pos,
            #                                     self.triple_neg: batch_neg,
            #                                     self.margin: [self.margin_value] * len(batch_pos)})

            #print(summary)
            #summary = tf.compat.v1.summary.merge(summary)
            #print(self.global_step.eval(session=session))
            #summary_writer.add_summary(tf.compat.v1.summary.merge(summary), global_step=self.global_step.eval(session=session))
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_head_embedding = self.entity_head_embedding.eval(session=session)
        entity_tail_embedding = self.entity_tail_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        head_entity_norm = np.linalg.norm(entity_head_embedding, ord=2, axis=1)
        tail_entity_norm = np.linalg.norm(entity_tail_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)
        #print(entity_norm)
        #print(entity_embedding)
        # entity_embedding_list = []
        # entity_embedding_norm_list = []
        # entity_embedding_list.append(entity_embedding)
        # entity_embedding_norm_list.append(entity_norm)

        with open('imdb30_c1_head_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_head_embedding)

        with open('imdb30_c1_tail_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_tail_embedding)

        with open('imdb30_c1_relation_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)

        # with open('List_norm1.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(entity_norm)

        #return entity_embedding_list #, entity_embedding_norm_list
        
        #print('entity norm: {} relation norm: {}'.format(entity_norm, relation_norm))

    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())

    #     # tf.train.Saver를 이용해서 모델과 파라미터를 저장합니다.
    # def save(self, session, data_dir):
    #   SAVER_DIR = "model"
    #   saver = tf.compat.v1.train.Saver()
    #   checkpoint_path = os.path.join(SAVER_DIR, "model")
    #   ckpt = tf.compat.v1.train.get_checkpoint_state(SAVER_DIR)



def main():
    #parser = argparse.ArgumentParser(description='TransE')
    # parser.add_argument('--data_dir', type=str, default='../data/FB15k/')
    # parser.add_argument('--embedding_dim', type=int, default=200)
    # parser.add_argument('--margin_value', type=float, default=1.0)
    # parser.add_argument('--score_func', type=str, default='L1')
    # parser.add_argument('--batch_size', type=int, default=4800)
    # parser.add_argument('--learning_rate', type=float, default=0.001)
    # parser.add_argument('--n_generator', type=int, default=24)
    # parser.add_argument('--n_rank_calculator', type=int, default=24)
    # parser.add_argument('--ckpt_dir', type=str, default='../ckpt/')
    # parser.add_argument('--summary_dir', type=str, default='../summary/')
    # parser.add_argument('--max_epoch', type=int, default=500)
    # parser.add_argument('--eval_freq', type=int, default=10)
    #args = parser.parse_args()
    #print(args)
    data_dir = ("/content/")
    embedding_dim = 50
    margin_value = 1.0
    score_func = "L1"
    batch_size =4800
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() 

### Embedding 100

In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity_head = 0
        self.n_entity_tail = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_head_dict_file = 'entity2id_head.txt'
        entity_tail_dict_file = 'entity2id_tail.txt'
        relation_dict_file = 'relation2id.txt'
        print('-----Loading entity dict-----')
        #entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= '\t')
        entity_head_df = pd.read_table(os.path.join(self.data_dir, entity_head_dict_file), header=None, sep= '\t')
        entity_tail_df = pd.read_table(os.path.join(self.data_dir, entity_tail_dict_file), header=None, sep= '\t')        
        print(entity_head_df.head())
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        #self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
        self.entity_head_dict = dict(zip(entity_head_df[0], entity_head_df[1]))
        self.entity_tail_dict = dict(zip(entity_tail_df[0], entity_tail_df[1]))        
        self.n_entity_head = len(self.entity_head_dict)
        self.n_entity_tail = len(self.entity_tail_dict)
        #print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        
        print(self.n_entity_head)
        print(self.n_entity_tail)
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= '\t')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        print(self.relation_dict)
        self.n_relation = len(self.relation_dict)
        print(self.n_relation)
        print('#relation: {}'.format(self.n_relation))
        # with open('label_entity_embedding.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'imdb30_train - imdb30_train.csv'
        validation_file = 'imdb30_valid - imdb30_valid.csv'
        test_file = 'imdb30_test - imdb30_test.csv'
        print('-----Loading training triples-----')
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= ',')
        print(training_df.head())
        self.training_triples = list(zip([self.entity_head_dict[h] for h in training_df[0]],
                                         [self.entity_tail_dict[t] for t in training_df[1]],
                                         [self.relation_dict[r] for r in training_df[2]]))
        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        print('-----Loading validation triples-----')
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= ',')
        self.validation_triples = list(zip([self.entity_head_dict[h] for h in validation_df[0]],
                                           [self.entity_tail_dict[t] for t in validation_df[1]],
                                           [self.relation_dict[r] for r in validation_df[2]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= ',')
        self.test_triples = list(zip([self.entity_head_dict[h] for h in test_df[0]],
                                     [self.entity_tail_dict[t] for t in test_df[1]],
                                     [self.relation_dict[r] for r in test_df[2]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        print("---------check-------------------")
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_head_embedding = tf.compat.v1.get_variable(name='entity_head',
                                                    shape=[kg.n_entity_head, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            self.entity_tail_embedding = tf.compat.v1.get_variable(name='entity_tail',
                                                    shape=[kg.n_entity_tail, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            # self.entity_embedding = tf.compat.v1.get_variable(name='entity',
            #                                         shape=[kg.n_entity, self.embedding_dim],
            #                                         initializer=tf.random_uniform_initializer(minval=-bound,
            #                                                                                   maxval=bound))
            print("----------check2--------------------")
            tf.summary.histogram('self.entity_head_embedding.op.name', self.entity_head_embedding)
            tf.summary.histogram('self.entity_tail_embedding.op.name', self.entity_tail_embedding)
            
            #tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity_head)
            print(kg.n_entity_tail)           
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            #self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.entity_head_embedding = tf.nn.l2_normalize(self.entity_head_embedding, dim=1)
            self.entity_tail_embedding = tf.nn.l2_normalize(self.entity_tail_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            head_pos = tf.nn.embedding_lookup(self.entity_head_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_head_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_head_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_tail_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_head_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_tail_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
        #print(n_batch)
        # print("merge!!!!!!!!!!!:",self.merge)
        # print("loss!!!!!!!!!!!:",self.loss)
        # print("train_op!!!!!!!!!!!:",self.train_op)
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
            #print("check2")
    
            # batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.merge],
            #                          feed_dict={self.triple_pos: batch_pos,
            #                                     self.triple_neg: batch_neg,
            #                                     self.margin: [self.margin_value] * len(batch_pos)})

            #print(summary)
            #summary = tf.compat.v1.summary.merge(summary)
            #print(self.global_step.eval(session=session))
            #summary_writer.add_summary(tf.compat.v1.summary.merge(summary), global_step=self.global_step.eval(session=session))
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_head_embedding = self.entity_head_embedding.eval(session=session)
        entity_tail_embedding = self.entity_tail_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        head_entity_norm = np.linalg.norm(entity_head_embedding, ord=2, axis=1)
        tail_entity_norm = np.linalg.norm(entity_tail_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)
        #print(entity_norm)
        #print(entity_embedding)
        # entity_embedding_list = []
        # entity_embedding_norm_list = []
        # entity_embedding_list.append(entity_embedding)
        # entity_embedding_norm_list.append(entity_norm)

        with open('imdb30_c1_head_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_head_embedding)

        with open('imdb30_c1_tail_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_tail_embedding)

        with open('imdb30_c1_relation_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)

        # with open('List_norm1.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(entity_norm)

        #return entity_embedding_list #, entity_embedding_norm_list
        
        #print('entity norm: {} relation norm: {}'.format(entity_norm, relation_norm))

    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())

    #     # tf.train.Saver를 이용해서 모델과 파라미터를 저장합니다.
    # def save(self, session, data_dir):
    #   SAVER_DIR = "model"
    #   saver = tf.compat.v1.train.Saver()
    #   checkpoint_path = os.path.join(SAVER_DIR, "model")
    #   ckpt = tf.compat.v1.train.get_checkpoint_state(SAVER_DIR)



def main():

    data_dir = ("/content/")
    embedding_dim = 100
    margin_value = 1.0
    score_func = "L1"
    batch_size =4800
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() 

## Trimmed Network based on analysis

### Embedding 50

In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity_head = 0
        self.n_entity_tail = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_head_dict_file = 'entity2id_head.txt'
        entity_tail_dict_file = 'entity2id_tail.txt'
        relation_dict_file = 'relation2id.txt'
        print('-----Loading entity dict-----')
        #entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= '\t')
        entity_head_df = pd.read_table(os.path.join(self.data_dir, entity_head_dict_file), header=None, sep= '\t')
        entity_tail_df = pd.read_table(os.path.join(self.data_dir, entity_tail_dict_file), header=None, sep= '\t')        
        print(entity_head_df.head())
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        #self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
        self.entity_head_dict = dict(zip(entity_head_df[0], entity_head_df[1]))
        self.entity_tail_dict = dict(zip(entity_tail_df[0], entity_tail_df[1]))        
        self.n_entity_head = len(self.entity_head_dict)
        self.n_entity_tail = len(self.entity_tail_dict)
        #print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        
        print(self.n_entity_head)
        print(self.n_entity_tail)
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= '\t')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        print(self.relation_dict)
        self.n_relation = len(self.relation_dict)
        print(self.n_relation)
        print('#relation: {}'.format(self.n_relation))
        # with open('label_entity_embedding.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'imdb30_train - imdb30_train.csv'
        validation_file = 'imdb30_valid - imdb30_valid.csv'
        test_file = 'imdb30_test - imdb30_test.csv'
        print('-----Loading training triples-----')
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= ',')
        print(training_df.head())
        self.training_triples = list(zip([self.entity_head_dict[h] for h in training_df[0]],
                                         [self.entity_tail_dict[t] for t in training_df[1]],
                                         [self.relation_dict[r] for r in training_df[2]]))
        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        print('-----Loading validation triples-----')
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= ',')
        self.validation_triples = list(zip([self.entity_head_dict[h] for h in validation_df[0]],
                                           [self.entity_tail_dict[t] for t in validation_df[1]],
                                           [self.relation_dict[r] for r in validation_df[2]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= ',')
        self.test_triples = list(zip([self.entity_head_dict[h] for h in test_df[0]],
                                     [self.entity_tail_dict[t] for t in test_df[1]],
                                     [self.relation_dict[r] for r in test_df[2]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        print("---------check-------------------")
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_head_embedding = tf.compat.v1.get_variable(name='entity_head',
                                                    shape=[kg.n_entity_head, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            self.entity_tail_embedding = tf.compat.v1.get_variable(name='entity_tail',
                                                    shape=[kg.n_entity_tail, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            # self.entity_embedding = tf.compat.v1.get_variable(name='entity',
            #                                         shape=[kg.n_entity, self.embedding_dim],
            #                                         initializer=tf.random_uniform_initializer(minval=-bound,
            #                                                                                   maxval=bound))
            print("----------check2--------------------")
            tf.summary.histogram('self.entity_head_embedding.op.name', self.entity_head_embedding)
            tf.summary.histogram('self.entity_tail_embedding.op.name', self.entity_tail_embedding)
            
            #tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity_head)
            print(kg.n_entity_tail)           
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            #self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.entity_head_embedding = tf.nn.l2_normalize(self.entity_head_embedding, dim=1)
            self.entity_tail_embedding = tf.nn.l2_normalize(self.entity_tail_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            head_pos = tf.nn.embedding_lookup(self.entity_head_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_head_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_head_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_tail_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_head_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_tail_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
        #print(n_batch)
        # print("merge!!!!!!!!!!!:",self.merge)
        # print("loss!!!!!!!!!!!:",self.loss)
        # print("train_op!!!!!!!!!!!:",self.train_op)
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
            #print("check2")
    
            # batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.merge],
            #                          feed_dict={self.triple_pos: batch_pos,
            #                                     self.triple_neg: batch_neg,
            #                                     self.margin: [self.margin_value] * len(batch_pos)})

            #print(summary)
            #summary = tf.compat.v1.summary.merge(summary)
            #print(self.global_step.eval(session=session))
            #summary_writer.add_summary(tf.compat.v1.summary.merge(summary), global_step=self.global_step.eval(session=session))
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_head_embedding = self.entity_head_embedding.eval(session=session)
        entity_tail_embedding = self.entity_tail_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        head_entity_norm = np.linalg.norm(entity_head_embedding, ord=2, axis=1)
        tail_entity_norm = np.linalg.norm(entity_tail_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)
        #print(entity_norm)
        #print(entity_embedding)
        # entity_embedding_list = []
        # entity_embedding_norm_list = []
        # entity_embedding_list.append(entity_embedding)
        # entity_embedding_norm_list.append(entity_norm)

        with open('imdb30_c1_head_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_head_embedding)

        with open('imdb30_c1_tail_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_tail_embedding)

        with open('imdb30_c1_relation_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)

        # with open('List_norm1.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(entity_norm)

        #return entity_embedding_list #, entity_embedding_norm_list
        
        #print('entity norm: {} relation norm: {}'.format(entity_norm, relation_norm))

    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())

    #     # tf.train.Saver를 이용해서 모델과 파라미터를 저장합니다.
    # def save(self, session, data_dir):
    #   SAVER_DIR = "model"
    #   saver = tf.compat.v1.train.Saver()
    #   checkpoint_path = os.path.join(SAVER_DIR, "model")
    #   ckpt = tf.compat.v1.train.get_checkpoint_state(SAVER_DIR)



def main():
    #parser = argparse.ArgumentParser(description='TransE')
    # parser.add_argument('--data_dir', type=str, default='../data/FB15k/')
    # parser.add_argument('--embedding_dim', type=int, default=200)
    # parser.add_argument('--margin_value', type=float, default=1.0)
    # parser.add_argument('--score_func', type=str, default='L1')
    # parser.add_argument('--batch_size', type=int, default=4800)
    # parser.add_argument('--learning_rate', type=float, default=0.001)
    # parser.add_argument('--n_generator', type=int, default=24)
    # parser.add_argument('--n_rank_calculator', type=int, default=24)
    # parser.add_argument('--ckpt_dir', type=str, default='../ckpt/')
    # parser.add_argument('--summary_dir', type=str, default='../summary/')
    # parser.add_argument('--max_epoch', type=int, default=500)
    # parser.add_argument('--eval_freq', type=int, default=10)
    #args = parser.parse_args()
    #print(args)
    data_dir = ("/content/")
    embedding_dim = 50
    margin_value = 1.0
    score_func = "L1"
    batch_size =4800
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() 

### Embedding 100

In [ ]:
class KnowledgeGraph:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.entity_dict = {}
        self.entities = []
        self.relation_dict = {}
        self.n_entity_head = 0
        self.n_entity_tail = 0
        self.n_relation = 0
        self.training_triples = []  # list of triples in the form of (h, t, r)
        self.validation_triples = []
        self.test_triples = []
        self.n_training_triple = 0
        self.n_validation_triple = 0
        self.n_test_triple = 0
        '''load dicts and triples'''
        self.load_dicts()
        self.load_triples()
        '''construct pools after loading'''
        self.training_triple_pool = set(self.training_triples)
        self.golden_triple_pool = set(self.training_triples) | set(self.validation_triples) | set(self.test_triples)

    def load_dicts(self):
        entity_head_dict_file = 'entity2id_head.txt'
        entity_tail_dict_file = 'entity2id_tail.txt'
        relation_dict_file = 'relation2id.txt'
        print('-----Loading entity dict-----')
        #entity_df = pd.read_table(os.path.join(self.data_dir, entity_dict_file), header=None, sep= '\t')
        entity_head_df = pd.read_table(os.path.join(self.data_dir, entity_head_dict_file), header=None, sep= '\t')
        entity_tail_df = pd.read_table(os.path.join(self.data_dir, entity_tail_dict_file), header=None, sep= '\t')        
        print(entity_head_df.head())
        #print(entity_df)
        #print(entity_df.info())
        #print(entity_df[0])
        #print(entity_df[1])
        #print(dict(entity_df[0], entity_df[1]))
        #self.entity_dict = dict(zip(entity_df[0], entity_df[1]))
        self.entity_head_dict = dict(zip(entity_head_df[0], entity_head_df[1]))
        self.entity_tail_dict = dict(zip(entity_tail_df[0], entity_tail_df[1]))        
        self.n_entity_head = len(self.entity_head_dict)
        self.n_entity_tail = len(self.entity_tail_dict)
        #print(self.n_entity)
        self.entities = list(self.entity_dict.values())
        #print(self.entities)
       # print('#entity: {}'.format(self.n_entity))
        
        print(self.n_entity_head)
        print(self.n_entity_tail)
        print('-----Loading relation dict-----')
        relation_df = pd.read_table(os.path.join(self.data_dir, relation_dict_file), header=None, sep= '\t')
        self.relation_dict = dict(zip(relation_df[0], relation_df[1]))
        print(self.relation_dict)
        self.n_relation = len(self.relation_dict)
        print(self.n_relation)
        print('#relation: {}'.format(self.n_relation))
        # with open('label_entity_embedding.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(self.entity_dict)


    def load_triples(self):
        training_file = 'imdb30_train - imdb30_train.csv'
        validation_file = 'imdb30_valid - imdb30_valid.csv'
        test_file = 'imdb30_test - imdb30_test.csv'
        print('-----Loading training triples-----')
        training_df = pd.read_table(os.path.join(self.data_dir, training_file), header=None, sep= ',')
        print(training_df.head())
        self.training_triples = list(zip([self.entity_head_dict[h] for h in training_df[0]],
                                         [self.entity_tail_dict[t] for t in training_df[1]],
                                         [self.relation_dict[r] for r in training_df[2]]))
        self.n_training_triple = len(self.training_triples)
        print('#training triple: {}'.format(self.n_training_triple))
        print('-----Loading validation triples-----')
        validation_df = pd.read_table(os.path.join(self.data_dir, validation_file), header=None, sep= ',')
        self.validation_triples = list(zip([self.entity_head_dict[h] for h in validation_df[0]],
                                           [self.entity_tail_dict[t] for t in validation_df[1]],
                                           [self.relation_dict[r] for r in validation_df[2]]))
        self.n_validation_triple = len(self.validation_triples)
        print('#validation triple: {}'.format(self.n_validation_triple))
        print('-----Loading test triples------')
        test_df = pd.read_table(os.path.join(self.data_dir, test_file), header=None, sep= ',')
        self.test_triples = list(zip([self.entity_head_dict[h] for h in test_df[0]],
                                     [self.entity_tail_dict[t] for t in test_df[1]],
                                     [self.relation_dict[r] for r in test_df[2]]))
        self.n_test_triple = len(self.test_triples)
        print('#test triple: {}'.format(self.n_test_triple))

    def next_raw_batch(self, batch_size):
        rand_idx = np.random.permutation(self.n_training_triple)
        start = 0
        while start < self.n_training_triple:
            end = min(start + batch_size, self.n_training_triple)
            yield [self.training_triples[i] for i in rand_idx[start:end]]
            start = end

    def generate_training_batch(self, in_queue, out_queue):
        while True:
            raw_batch = in_queue.get()
            if raw_batch is None:
                return
            else:
                batch_pos = raw_batch
                batch_neg = []
                corrupt_head_prob = np.random.binomial(1, 0.5)
                for head, tail, relation in batch_pos:
                    head_neg = head
                    tail_neg = tail
                    while True:
                        if corrupt_head_prob:
                            head_neg = random.choice(self.entities)
                        else:
                            tail_neg = random.choice(self.entities)
                        if (head_neg, tail_neg, relation) not in self.training_triple_pool:
                            break
                    batch_neg.append((head_neg, tail_neg, relation))
                out_queue.put((batch_pos, batch_neg))




import math
import timeit
import numpy as np
import tensorflow as tf
import multiprocessing as mp
import csv
#from dataset import KnowledgeGraph


class TransE:
    def __init__(self, kg: KnowledgeGraph,
                 embedding_dim, margin_value, score_func,
                 batch_size, learning_rate, n_generator, n_rank_calculator):
        self.kg = kg
        self.embedding_dim = embedding_dim
        self.margin_value = margin_value
        self.score_func = score_func
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_generator = n_generator
        self.n_rank_calculator = n_rank_calculator
        '''ops for training'''
        self.triple_pos = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.triple_neg = tf.compat.v1.placeholder(dtype=tf.int32, shape=[None, 3])
        self.margin = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.train_op = None
        #self.train_op = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None])
        self.loss = None
        self.global_step = tf.Variable(initial_value=0, trainable=False, name='global_step')
        self.merge = None
        '''ops for evaluation'''
        self.eval_triple = tf.compat.v1.placeholder(dtype=tf.int32, shape=[3])
        self.idx_head_prediction = None
        self.idx_tail_prediction = None
        '''embeddings'''
        bound = 6 / math.sqrt(self.embedding_dim)
        print("---------check-------------------")
        with tf.compat.v1.variable_scope('embedding'):
            self.entity_head_embedding = tf.compat.v1.get_variable(name='entity_head',
                                                    shape=[kg.n_entity_head, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            self.entity_tail_embedding = tf.compat.v1.get_variable(name='entity_tail',
                                                    shape=[kg.n_entity_tail, self.embedding_dim],
                                                    initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                              maxval=bound))
            # self.entity_embedding = tf.compat.v1.get_variable(name='entity',
            #                                         shape=[kg.n_entity, self.embedding_dim],
            #                                         initializer=tf.random_uniform_initializer(minval=-bound,
            #                                                                                   maxval=bound))
            print("----------check2--------------------")
            tf.summary.histogram('self.entity_head_embedding.op.name', self.entity_head_embedding)
            tf.summary.histogram('self.entity_tail_embedding.op.name', self.entity_tail_embedding)
            
            #tf.summary.histogram('self.entity_embedding.op.name', self.entity_embedding)
            #tf.summary.histogram(name=self.entity_embedding.op.name, values=self.entity_embedding)

            print(kg.n_entity_head)
            print(kg.n_entity_tail)           
            
            #tf.summary.histogram('SampleKernel', tf.random.normal(shape=[3,3,3,32], dtype=tf.float32))
            self.relation_embedding = tf.compat.v1.get_variable(name='relation',
                                                      shape=[kg.n_relation, self.embedding_dim],
                                                      initializer=tf.random_uniform_initializer(minval=-bound,
                                                                                                maxval=bound))
            tf.summary.histogram('self.relation_embedding.op.name', self.relation_embedding)
            #tf.summary.histogram(name=self.relation_embedding.op.name, values=self.relation_embedding)
        self.build_graph()
        self.build_eval_graph()

    def build_graph(self):
        with tf.name_scope('normalization'):
            #self.entity_embedding = tf.nn.l2_normalize(self.entity_embedding, dim=1)
            self.entity_head_embedding = tf.nn.l2_normalize(self.entity_head_embedding, dim=1)
            self.entity_tail_embedding = tf.nn.l2_normalize(self.entity_tail_embedding, dim=1)
            self.relation_embedding = tf.nn.l2_normalize(self.relation_embedding, dim=1)
        with tf.name_scope('training'):
            distance_pos, distance_neg = self.infer(self.triple_pos, self.triple_neg)
            self.loss = self.calculate_loss(distance_pos, distance_neg, self.margin)
            #tf.summary.scalar(name=self.loss.op.name, tensor=self.loss)
            tf.summary.scalar('self.loss.op.name', self.loss)
            optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step)
            #self.merge = tf.compat.v1.summary.merge_all()
            #self.merge = tf.compat.v1.summary.merge_all(key=_ops.GraphKeys.SUMMARIES, scope=None, name=None)
            #self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES, scope=None, name=None)
            self.merge = tf.compat.v1.summary.merge_all(key = _ops.GraphKeys.SUMMARIES)
 
            tf.compat.v1.summary.merge_all()
            #tf.merge_all_summaries()

    def build_eval_graph(self):
        with tf.name_scope('evaluation'):
            self.idx_head_prediction, self.idx_tail_prediction = self.evaluate(self.eval_triple)

    def infer(self, triple_pos, triple_neg):
        with tf.name_scope('lookup'):
            head_pos = tf.nn.embedding_lookup(self.entity_head_embedding, triple_pos[:, 0])
            tail_pos = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_pos[:, 1])
            relation_pos = tf.nn.embedding_lookup(self.relation_embedding, triple_pos[:, 2])
            head_neg = tf.nn.embedding_lookup(self.entity_head_embedding, triple_neg[:, 0])
            tail_neg = tf.nn.embedding_lookup(self.entity_tail_embedding, triple_neg[:, 1])
            relation_neg = tf.nn.embedding_lookup(self.relation_embedding, triple_neg[:, 2])
        with tf.name_scope('link'):
            distance_pos = head_pos + relation_pos - tail_pos
            distance_neg = head_neg + relation_neg - tail_neg
        return distance_pos, distance_neg

    def calculate_loss(self, distance_pos, distance_neg, margin):
        with tf.name_scope('loss'):
            if self.score_func == 'L1':  # L1 score
                score_pos = tf.reduce_sum(tf.abs(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.abs(distance_neg), axis=1)
            else:  # L2 score
                score_pos = tf.reduce_sum(tf.square(distance_pos), axis=1)
                score_neg = tf.reduce_sum(tf.square(distance_neg), axis=1)
            loss = tf.reduce_sum(tf.nn.relu(margin + score_pos - score_neg), name='max_margin_loss')
        return loss

    def evaluate(self, eval_triple):
        with tf.name_scope('lookup'):
            head = tf.nn.embedding_lookup(self.entity_head_embedding, eval_triple[0])
            tail = tf.nn.embedding_lookup(self.entity_tail_embedding, eval_triple[1])
            relation = tf.nn.embedding_lookup(self.relation_embedding, eval_triple[2])
        with tf.name_scope('link'):
            distance_head_prediction = self.entity_head_embedding + relation - tail
            distance_tail_prediction = head + relation - self.entity_tail_embedding
        with tf.name_scope('rank'):
            if self.score_func == 'L1':  # L1 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.abs(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
            else:  # L2 score
                _, idx_head_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_head_prediction), axis=1),
                                                     k=self.kg.n_entity_head)
                _, idx_tail_prediction = tf.nn.top_k(tf.reduce_sum(tf.square(distance_tail_prediction), axis=1),
                                                     k=self.kg.n_entity_tail)
        return idx_head_prediction, idx_tail_prediction

    def launch_training(self, session, summary_writer):
        raw_batch_queue = mp.Queue()
        training_batch_queue = mp.Queue()
        for _ in range(self.n_generator):
            mp.Process(target=self.kg.generate_training_batch, kwargs={'in_queue': raw_batch_queue,
                                                                       'out_queue': training_batch_queue}).start()
        print('-----Start training-----')
        start = timeit.default_timer()
        n_batch = 0
        for raw_batch in self.kg.next_raw_batch(self.batch_size):
            raw_batch_queue.put(raw_batch)
            n_batch += 1
        for _ in range(self.n_generator):
            raw_batch_queue.put(raw_batch)
            #raw_batch_queue.put(None)
        print('-----Constructing training batches-----')
        epoch_loss = 0
        n_used_triple = 0
        #print(n_batch)
        # print("merge!!!!!!!!!!!:",self.merge)
        # print("loss!!!!!!!!!!!:",self.loss)
        # print("train_op!!!!!!!!!!!:",self.train_op)
        
        for i in range(n_batch):
            batch_pos, batch_neg = training_batch_queue.get()
            #print("check1")
            batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.margin ],
                                                 feed_dict={self.triple_pos: batch_pos,
                                                            self.triple_neg: batch_neg,
                                                            self.margin: [self.margin_value] * len(batch_pos)})
            #print("check2")
    
            # batch_loss, _, summary = session.run(fetches=[self.loss, self.train_op, self.merge],
            #                          feed_dict={self.triple_pos: batch_pos,
            #                                     self.triple_neg: batch_neg,
            #                                     self.margin: [self.margin_value] * len(batch_pos)})

            #print(summary)
            #summary = tf.compat.v1.summary.merge(summary)
            #print(self.global_step.eval(session=session))
            #summary_writer.add_summary(tf.compat.v1.summary.merge(summary), global_step=self.global_step.eval(session=session))
            summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #summary_writer.add_summary(summary, global_step=self.global_step.eval(session=session))
            #print("check3")
            epoch_loss += batch_loss
            n_used_triple += len(batch_pos)
            #print("check4")  
            #print("batch_op")
            print('[{:.3f}s] #triple: {}/{} triple_avg_loss: {:.6f}'.format(timeit.default_timer() - start,
                                                                            n_used_triple,
                                                                            self.kg.n_training_triple,
                                                                            batch_loss / len(batch_pos)), end='\r')
        print()
        print('epoch loss: {:.3f}'.format(epoch_loss))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish training-----')
        self.check_norm(session=session)
        #return self.kg.n_training_triple

    def launch_evaluation(self, session):
        eval_result_queue = mp.JoinableQueue()
        rank_result_queue = mp.Queue()
        print('-----Start evaluation-----')
        start = timeit.default_timer()
        for _ in range(self.n_rank_calculator):
            mp.Process(target=self.calculate_rank, kwargs={'in_queue': eval_result_queue,
                                                           'out_queue': rank_result_queue}).start()
        n_used_eval_triple = 0
        for eval_triple in self.kg.test_triples:
            idx_head_prediction, idx_tail_prediction = session.run(fetches=[self.idx_head_prediction,
                                                                            self.idx_tail_prediction],
                                                                   feed_dict={self.eval_triple: eval_triple})
            eval_result_queue.put((eval_triple, idx_head_prediction, idx_tail_prediction))
            n_used_eval_triple += 1
            print('[{:.3f}s] #evaluation triple: {}/{}'.format(timeit.default_timer() - start,
                                                               n_used_eval_triple,
                                                               self.kg.n_test_triple), end='\r')
        print()
        for _ in range(self.n_rank_calculator):
            eval_result_queue.put(None)
        print('-----Joining all rank calculator-----')
        eval_result_queue.join()
        print('-----All rank calculation accomplished-----')
        print('-----Obtaining evaluation results-----')
        '''Raw'''
        head_meanrank_raw = 0
        head_hits10_raw = 0
        tail_meanrank_raw = 0
        tail_hits10_raw = 0
        '''Filter'''
        head_meanrank_filter = 0
        head_hits10_filter = 0
        tail_meanrank_filter = 0
        tail_hits10_filter = 0
        for _ in range(n_used_eval_triple):
            head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter = rank_result_queue.get()
            head_meanrank_raw += head_rank_raw
            if head_rank_raw < 10:
                head_hits10_raw += 1
            tail_meanrank_raw += tail_rank_raw
            if tail_rank_raw < 10:
                tail_hits10_raw += 1
            head_meanrank_filter += head_rank_filter
            if head_rank_filter < 10:
                head_hits10_filter += 1
            tail_meanrank_filter += tail_rank_filter
            if tail_rank_filter < 10:
                tail_hits10_filter += 1
        print('-----Raw-----')
        head_meanrank_raw /= n_used_eval_triple
        head_hits10_raw /= n_used_eval_triple
        tail_meanrank_raw /= n_used_eval_triple
        tail_hits10_raw /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_raw, head_hits10_raw))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_raw, tail_hits10_raw))
        print('------Average------')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_raw + tail_meanrank_raw) / 2,
                                                         (head_hits10_raw + tail_hits10_raw) / 2))
        print('-----Filter-----')
        head_meanrank_filter /= n_used_eval_triple
        head_hits10_filter /= n_used_eval_triple
        tail_meanrank_filter /= n_used_eval_triple
        tail_hits10_filter /= n_used_eval_triple
        print('-----Head prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(head_meanrank_filter, head_hits10_filter))
        print('-----Tail prediction-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format(tail_meanrank_filter, tail_hits10_filter))
        print('-----Average-----')
        print('MeanRank: {:.3f}, Hits@10: {:.3f}'.format((head_meanrank_filter + tail_meanrank_filter) / 2,
                                                         (head_hits10_filter + tail_hits10_filter) / 2))
        print('cost time: {:.3f}s'.format(timeit.default_timer() - start))
        print('-----Finish evaluation-----')

    def calculate_rank(self, in_queue, out_queue):
        while True:
            idx_predictions = in_queue.get()
            if idx_predictions is None:
                in_queue.task_done()
                return
            else:
                eval_triple, idx_head_prediction, idx_tail_prediction = idx_predictions
                head, tail, relation = eval_triple
                head_rank_raw = 0
                tail_rank_raw = 0
                head_rank_filter = 0
                tail_rank_filter = 0
                for candidate in idx_head_prediction[::-1]:
                    if candidate == head:
                        break
                    else:
                        head_rank_raw += 1
                        if (candidate, tail, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            head_rank_filter += 1
                for candidate in idx_tail_prediction[::-1]:
                    if candidate == tail:
                        break
                    else:
                        tail_rank_raw += 1
                        if (head, candidate, relation) in self.kg.golden_triple_pool:
                            continue
                        else:
                            tail_rank_filter += 1
                out_queue.put((head_rank_raw, tail_rank_raw, head_rank_filter, tail_rank_filter))
                in_queue.task_done()

    def check_norm(self, session):
        print('-----Check norm-----')
        entity_head_embedding = self.entity_head_embedding.eval(session=session)
        entity_tail_embedding = self.entity_tail_embedding.eval(session=session)
        relation_embedding = self.relation_embedding.eval(session=session)
        head_entity_norm = np.linalg.norm(entity_head_embedding, ord=2, axis=1)
        tail_entity_norm = np.linalg.norm(entity_tail_embedding, ord=2, axis=1)
        relation_norm = np.linalg.norm(relation_embedding, ord=2, axis=1)
        #print(entity_norm)
        #print(entity_embedding)
        # entity_embedding_list = []
        # entity_embedding_norm_list = []
        # entity_embedding_list.append(entity_embedding)
        # entity_embedding_norm_list.append(entity_norm)

        with open('imdb30_c1_head_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_head_embedding)

        with open('imdb30_c1_tail_entity_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(entity_tail_embedding)

        with open('imdb30_c1_relation_embedding_100.csv','w') as file :
          write = csv.writer(file)
          write.writerows(relation_embedding)

        # with open('List_norm1.csv','w') as file :
        #   write = csv.writer(file)
        #   write.writerows(entity_norm)

        #return entity_embedding_list #, entity_embedding_norm_list
        
        #print('entity norm: {} relation norm: {}'.format(entity_norm, relation_norm))

    def save(self, session, data_dir):
        saver = tf.compat.v1.train.Saver()
        saver.save(session, data_dir, global_step=self.global_step.eval())

    #     # tf.train.Saver를 이용해서 모델과 파라미터를 저장합니다.
    # def save(self, session, data_dir):
    #   SAVER_DIR = "model"
    #   saver = tf.compat.v1.train.Saver()
    #   checkpoint_path = os.path.join(SAVER_DIR, "model")
    #   ckpt = tf.compat.v1.train.get_checkpoint_state(SAVER_DIR)



def main():
    #parser = argparse.ArgumentParser(description='TransE')
    # parser.add_argument('--data_dir', type=str, default='../data/FB15k/')
    # parser.add_argument('--embedding_dim', type=int, default=200)
    # parser.add_argument('--margin_value', type=float, default=1.0)
    # parser.add_argument('--score_func', type=str, default='L1')
    # parser.add_argument('--batch_size', type=int, default=4800)
    # parser.add_argument('--learning_rate', type=float, default=0.001)
    # parser.add_argument('--n_generator', type=int, default=24)
    # parser.add_argument('--n_rank_calculator', type=int, default=24)
    # parser.add_argument('--ckpt_dir', type=str, default='../ckpt/')
    # parser.add_argument('--summary_dir', type=str, default='../summary/')
    # parser.add_argument('--max_epoch', type=int, default=500)
    # parser.add_argument('--eval_freq', type=int, default=10)
    #args = parser.parse_args()
    #print(args)
    data_dir = ("/content/")
    embedding_dim = 100
    margin_value = 1.0
    score_func = "L1"
    batch_size =4800
    learning_rate = 0.01
    n_generator =24
    n_rank_calculator = 24
    ckpt_dir = '../ckpt/'
    summary_dir = '../summary/'
    max_epoch = 1000
    eval_freq = 10

     
    kg = KnowledgeGraph(data_dir=data_dir)
    kge_model = TransE(kg=kg, embedding_dim=embedding_dim, margin_value=margin_value,
                       score_func=score_func, batch_size=batch_size, learning_rate=learning_rate,
                       n_generator=n_generator, n_rank_calculator=n_rank_calculator)
    
    gpu_config = tf.compat.v1.GPUOptions(allow_growth=True)
    sess_config = tf.compat.v1.ConfigProto(gpu_options=gpu_config)
    with tf.compat.v1.Session(config=sess_config) as sess:
        print('-----Initializing tf graph-----')
        tf.compat.v1.global_variables_initializer().run()
        print('-----Initialization accomplished-----')
        kge_model.check_norm(session=sess)
        summary_writer = tf.compat.v1.summary.FileWriter(logdir=summary_dir, graph=sess.graph)
        
        for epoch in range(max_epoch):
            print('=' * 30 + '[EPOCH {}]'.format(epoch) + '=' * 30)
            kge_model.launch_training(session=sess, summary_writer=summary_writer)
            if (epoch + 1) % eval_freq == 0:
                kge_model.launch_evaluation(session=sess)
        kge_model.save( session = sess, data_dir = data_dir)  
        print("---------save-----------")    
        #kge_model.check_norm(sess)
        #return entity_embedding_list, entity_embedding_norm_list

In [ ]:
main() 